# Analyze files per group of SK channel traces

Supplement to the manuscript 'Gating kinetics and pharmacological properties of small conductance calcium-activated potassium channels' by Ilsbeth G.M. van Herck, Vincent Seutin, Bo H. Bentzen, Neil V. Marrion and Andrew G. Edwards <br/>
DOI:

This code analyzes the single SK channel traces per group (control, apamin and AP14145). Results are presented as text under the corresponding code. <br/>
Part 1: Analyze all files for total time and events per group <br/>
Part 2: Analyze all single channel patches for open probability and burst frequency <br/>
Part 3: Analyze inter-burst behavior for selected patches <br/>

Note: Part 1 has to be executed to load all data and perform analysis in part 2 and 3. 

In [30]:
%reset -f
import os
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook

### Set output directory

In [31]:
outdir = "group_analysis"

### Choose group to analyze: control, apamin or AP14145

In [32]:
import ipywidgets
group_select = ipywidgets.Dropdown(options=['control','apamin','AP14145'], 
                    description='Group')

group_select

Dropdown(description='Group', options=('control', 'apamin', 'AP14145'), value='control')

## Part 1: Analyze all files for total time and events per group

In [33]:
group = group_select.label
if group == 'control':
    group_list_all = ['control_1', 'control_2', 'control_3', 'control_4','control_5','control_6','control_7','control_8','control_9','control_10','control_11','control_12','control_13','control_14','control_15','control_16_segment6','control_16_segment37']
    print("Number of recordings in group {}: {}".format(group, len(group_list_all)-1))
elif group == 'apamin':
    group_list_all = ['apamin_1', 'apamin_2', 'apamin_3', 'apamin_4','apamin_5','apamin_6','apamin_7','apamin_8','apamin_9','apamin_10','apamin_11']
    print("Number of recordings in group {}: {}".format(group, len(group_list_all)))
elif group == 'AP14145':
    group_list_all = ['AP14145_1', 'AP14145_2', 'AP14145_3', 'AP14145_4','AP14145_5','AP14145_6','AP14145_7','AP14145_8','AP14145_9']
    print("Number of recordings in group {}: {}".format(group, len(group_list_all)))

Number of recordings in group control: 16


In [34]:
##Create output directory
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [35]:
# Load event files

data_dict = {}
for patch in group_list_all:
    data_dict[patch] = {}
    
    print("loading patch: ", patch)
    eventdata = np.loadtxt(os.path.join('./data/' + patch))
    
    times_nonzero = eventdata[:,1]
    durations = np.diff(times_nonzero)
    levels = eventdata[:,4]
    
    #Save traces in a dictionary
    data_dict[patch].update({"time": times_nonzero, "durations": durations, "levels": levels})
    
    # Create array with level for each timepoint
    dt = 0.000001 #sec
    if os.path.isfile(os.path.join(outdir,patch+'_TimeLevel_dt.npy')):
        times = np.load(os.path.join(outdir,patch+'_TimeLevel_dt.npy'))
        timeframe = np.ceil(times[-1,0])
        print("Loaded 'times' from datafile")
    else:
        times_startzero = np.zeros((len(eventdata)))
        times_startzero[:] = times_nonzero - eventdata[0,1] #adjusting to start at 0

        timeframe = np.ceil(times_startzero[-1])
        times = np.zeros((int(timeframe/dt),2))

        try:
            times[:,0] = np.arange(0,timeframe,dt)
        except ValueError:
            times[:,0] = np.arange(0,timeframe-dt,dt)

        j=0
        for i in range(0,len(times)-1):
            if times[i,0] >= times_startzero[j] and times[i,0] < times_startzero[j+1]:
                times[i,1] = [0, levels[j]][levels[j] == -1]
            elif j == len(eventdata)-2:
                break
            else:
                j=j+1
                
        #save 'times' vector for later use. Saves waiting time, especially for longer experiments. 
        with open(os.path.join(outdir,patch+'_TimeLevel_dt.npy'), 'wb') as f:
            np.save(f, times)
    data_dict[patch].update({"time_dt":times[:,0], "level_dt":times[:,1]})

loading patch:  control_1
Loaded 'times' from datafile
loading patch:  control_2
Loaded 'times' from datafile
loading patch:  control_3
Loaded 'times' from datafile
loading patch:  control_4
Loaded 'times' from datafile
loading patch:  control_5
Loaded 'times' from datafile
loading patch:  control_6
Loaded 'times' from datafile
loading patch:  control_7
Loaded 'times' from datafile
loading patch:  control_8
Loaded 'times' from datafile
loading patch:  control_9
Loaded 'times' from datafile
loading patch:  control_10
Loaded 'times' from datafile
loading patch:  control_11
Loaded 'times' from datafile
loading patch:  control_12
Loaded 'times' from datafile
loading patch:  control_13
Loaded 'times' from datafile
loading patch:  control_14
Loaded 'times' from datafile
loading patch:  control_15
Loaded 'times' from datafile
loading patch:  control_16_segment6
Loaded 'times' from datafile
loading patch:  control_16_segment37
Loaded 'times' from datafile


In [36]:
# Calculate total duration and number of openings
totaldur_group = 0
number_events = 0
for patch in group_list_all:
    totaldur_group += data_dict[patch]["time"][-1] - data_dict[patch]["time"][0]
    number_events += np.floor(len(data_dict[patch]["levels"])/2)

print("Total duration of recording in group {} (s): {}".format(group, totaldur_group))
print("Total number of channel closure events: {}".format(number_events))

Total duration of recording in group control (s): 746.4038022699999
Total number of channel closure events: 10915.0


In [37]:
#Calculate number of bursts

def zero_runs(a):
    ##Find runs of 0's in an array
    # Create an array that is 1 where a is 0, and pad each end with an extra 0
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

def number_of_closed_periods(a, thresh): #Input is array of arrays from zero_runs()
    ##Find number of closed periods >thresh (ms) in trace 
    Q_runs = np.where(a[:,1] - a[:,0] > thresh)
    return len(Q_runs[0])
    
def get_indices_to_closed_periods(a, thresh): #Input is array of arrays from zero_runs()
    ##Get indices for start and end of closed period >thresh (ms)
    start_idx = []
    end_idx = []
    j=0
    for i,Qrun in enumerate(a):
        if Qrun[1] - Qrun[0] > thresh:
            start_idx.append(Qrun[0])
            end_idx.append(Qrun[1])
    return start_idx, end_idx

##Calculate time spent in burst
if group == "control":
    burst_delim = 31.8*1000 #ms to us
elif group == "apamin":
    burst_delim = 12.3*1000 #ms to us
elif group == "AP14145":
    burst_delim = 87.0*1000 #ms to us
else:
    print("Cannot assign burst delimiter. ")


bursts = 0
bursts_dur = []
for patch in group_list_all:
    #Find closed duration that split bursts
    runs = zero_runs(data_dict[patch]["level_dt"])
    number_burst = number_of_closed_periods(runs, burst_delim)
    bursts += number_burst
    print("Number of bursts for {}: {}".format(patch, number_burst))
    
    endburst_idx, startburst_idx = get_indices_to_closed_periods(runs, burst_delim)

    #Calculate burst duration
    idx = 0
    if startburst_idx[0] < endburst_idx[0]:
        burstdur = [ele1 - ele2 for (ele1, ele2) in zip(endburst_idx,startburst_idx)]
    else: 
        del endburst_idx[0]
        burstdur = [ele1 - ele2 for (ele1, ele2) in zip(endburst_idx,startburst_idx)]
    bursts_dur += burstdur
    if len(burstdur) != 0:
        print("Average burst duration (ms): ", sum(burstdur)/len(burstdur)/1000)
    else:
        print("Zero bursts in patch")
print("")
print("Total number of bursts for group: ", bursts)
print("Average burst duration for group (ms): ", sum(bursts_dur)/len(bursts_dur)/1000)

Number of bursts for control_1: 8
Average burst duration (ms):  2275.4351428571426
Number of bursts for control_2: 745
Average burst duration (ms):  41.451201612903226
Number of bursts for control_3: 105
Average burst duration (ms):  23.286778846153847
Number of bursts for control_4: 685
Average burst duration (ms):  17.06776023391813
Number of bursts for control_5: 20
Average burst duration (ms):  180.52042105263158
Number of bursts for control_6: 10
Average burst duration (ms):  807.1924444444445
Number of bursts for control_7: 17
Average burst duration (ms):  349.380625
Number of bursts for control_8: 46
Average burst duration (ms):  104.13015555555555
Number of bursts for control_9: 246
Average burst duration (ms):  27.90988163265306
Number of bursts for control_10: 63
Average burst duration (ms):  272.5406935483871
Number of bursts for control_11: 61
Average burst duration (ms):  181.06668333333332
Number of bursts for control_12: 11
Average burst duration (ms):  445.7009000000000

## Part 2: Analyze all single channel patches for Po and burst frequency

In [38]:
if group == 'control':
    group_list_1ch = ['control_1','control_2','control_3','control_4','control_12']
elif group == 'apamin':
    group_list_1ch = ['apamin_1','apamin_2','apamin_3','apamin_4']
elif group == 'AP14145':
    group_list_1ch = ['AP14145_1','AP14145_2','AP14145_3','AP14145_4','AP14145_6','AP14145_8']

In [39]:
# Calculate total duration of recordings in group
totaldur_group = 0
for patch in group_list_1ch:
    totaldur_group += data_dict[patch]["time"][-1] - data_dict[patch]["time"][0]

print("Total duration of recording in group {} (s): {}".format(group, totaldur_group))

Total duration of recording in group control (s): 372.26588604


In [40]:
# Calculate Po
open_dt = 0
all_level_length = 0
Po_patch = np.zeros(len(group_list_1ch))
for i, patch in enumerate(group_list_1ch):
    nonzeros = np.count_nonzero(data_dict[patch]["level_dt"])
    all_level_length += len(data_dict[patch]["level_dt"]) 
    Po_patch[i] = nonzeros/len(data_dict[patch]["level_dt"])
    print("Open probability (Po): {}".format(nonzeros/len(data_dict[patch]["level_dt"])))
print("")
print("Average open probability of entire group: ", sum(Po_patch)/len(Po_patch))
print("Open probability of entire group: ", nonzeros/all_level_length)

Open probability (Po): 0.897101
Open probability (Po): 0.15643943670886076
Open probability (Po): 0.0385108125
Open probability (Po): 0.0404306779661017
Open probability (Po): 0.45670533333333335

Average open probability of entire group:  0.31783745210165915
Open probability of entire group:  0.007326823529411765


In [41]:
# Calculate burst frequency
bursts = 0
bursts_dur = []
for patch in group_list_1ch:
    #Find closed duration that split bursts
    runs = zero_runs(data_dict[patch]["level_dt"])
    number_burst = number_of_closed_periods(runs, burst_delim)
    bursts += number_burst
    print("Number of bursts for {}: {}".format(patch, number_burst))
    
    endburst_idx, startburst_idx = get_indices_to_closed_periods(runs, burst_delim)

    #Calculate burst duration
    idx = 0
    if startburst_idx[0] < endburst_idx[0]:
        burstdur = [ele1 - ele2 for (ele1, ele2) in zip(endburst_idx,startburst_idx)]
    else: 
        del endburst_idx[0]
        burstdur = [ele1 - ele2 for (ele1, ele2) in zip(endburst_idx,startburst_idx)]
    bursts_dur += burstdur
    if len(burstdur) != 0:
        print("Average burst duration (ms): ", sum(burstdur)/len(burstdur)/1000)
    else:
        print("Zero bursts in patch")
print("")
print("Total number of bursts for group: ", bursts)
print("Average burst duration for group (ms): ", sum(bursts_dur)/len(bursts_dur)/1000)
print("Burst frequency (bursts/s): ", bursts/totaldur_group)

Number of bursts for control_1: 8
Average burst duration (ms):  2275.4351428571426
Number of bursts for control_2: 745
Average burst duration (ms):  41.451201612903226
Number of bursts for control_3: 105
Average burst duration (ms):  23.286778846153847
Number of bursts for control_4: 685
Average burst duration (ms):  17.06776023391813
Number of bursts for control_12: 11
Average burst duration (ms):  445.70090000000005

Total number of bursts for group:  1554
Average burst duration for group (ms):  42.16973660426081
Burst frequency (bursts/s):  4.174435687703661


In [42]:
#Calculate time with Po<0.1 or Po>0.8 (in %)
binsize = 0.1 #ms

totalbins = 0
nrbins_Po_lower_0p1 = 0
nrbins_Po_higher_0p8 = 0
#Calculate Po for each 100 dt-steps (dt=0.000001s=1us)
for patch in group_list_1ch:
    for i in range(0,len(data_dict[patch]["level_dt"]),100):
        totalbins += 1
        nonzeros = np.count_nonzero(data_dict[patch]["level_dt"][i:i+100])
        Po_bin = nonzeros/100
        if Po_bin < 0.1:
            nrbins_Po_lower_0p1 += 1
        if Po_bin > 0.8:
            nrbins_Po_higher_0p8 += 1

print("% bins with Po<0.1: ", nrbins_Po_lower_0p1*100/totalbins)
print("% bins with Po>0.8: ", nrbins_Po_higher_0p8*100/totalbins)

% bins with Po<0.1:  86.41764705882353
% bins with Po>0.8:  13.434064171122994


## Part 3: Analyze inter-burst behavior for selected patches

In [43]:
if group == 'control':
    group_list = ['control_1', 'control_2', 'control_3', 'control_4']
elif group == 'apamin':
    group_list = ['apamin_1', 'apamin_2', 'apamin_3', 'apamin_4']
elif group == 'AP14145':
    group_list = ['AP14145_1', 'AP14145_2', 'AP14145_3']

In [44]:
binsize = 0.1 #ms
dt = 0.000001 #s, results in dt=1us
data_dict = {}
open_dur_group = []
closed_dur_group = []

for patch in group_list:
    print("patch: ", patch)
    eventdata = np.loadtxt('data/' + patch)
    
    ## load data from event file
    #time = np.zeros((len(eventdata)))
    #level = np.zeros((len(eventdata)))
    time = eventdata[:,1] - eventdata[0,1] #adjusting to start at 0
    level = eventdata[:,4]
    
    all_dur = np.diff(time)
    open_dur = all_dur[0::2]
    closed_dur = all_dur[1::2]
    print("max open dur: ", max(open_dur))
    print("max closed dur: ", max(closed_dur))
    print("Po: ", sum(open_dur)/(sum(open_dur)+sum(closed_dur)))
    
    open_dur_group.append(open_dur)
    closed_dur_group.append(closed_dur)
    
    #Store results in dictionary
    data_dict.update({patch: {"time": time, "level":level, "timings": all_dur, \
                              "open_dur": open_dur, "closed_dur": closed_dur}})
    

patch:  control_1
max open dur:  0.3549825800000006
max closed dur:  0.0653552899999994
Po:  0.9125309388588866
patch:  control_2
max open dur:  0.20878509000000633
max closed dur:  2.687538419999999
Po:  0.15661436194677078
patch:  control_3
max open dur:  0.021921419999998193
max closed dur:  0.7123318699999999
Po:  0.039873424609884446
patch:  control_4
max open dur:  0.09436774999999997
max closed dur:  2.797506830000003
Po:  0.04053248222622269
